# Winery classification with the bivariate Gaussian

Our first generative model for Winery classification used just one feature. Now we use two features, modeling each class by a **bivariate Gaussian**.

## 1. Load in the data set

As in the univariate case, we start by loading in the Wine data set. Make sure the file `wine.data.txt` is in the same directory as this notebook.

Recall that there are 178 data points, each with 13 features and a label (1,2,3). As before, we will divide this into a training set of 130 points and a test set of 48 points.

In [2]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal 
# installing packages for interactive graphs
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

In [3]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

In [9]:
trainx[:,1]

array([1.67, 2.67, 1.13, 1.73, 5.8 , 2.05, 2.15, 3.83, 1.73, 1.83, 2.56,
       5.04, 1.76, 1.07, 2.58, 0.92, 1.53, 1.59, 1.36, 1.65, 1.01, 1.65,
       2.59, 1.73, 1.1 , 2.43, 1.53, 1.61, 2.68, 2.76, 1.77, 1.7 , 4.3 ,
       4.04, 1.64, 1.77, 3.26, 1.53, 1.99, 3.91, 1.81, 1.5 , 2.83, 1.34,
       0.74, 3.1 , 2.46, 1.09, 3.88, 1.92, 2.99, 3.59, 1.71, 1.68, 1.86,
       1.29, 4.28, 3.57, 1.51, 3.27, 4.12, 1.72, 3.86, 1.24, 1.41, 3.43,
       3.45, 1.25, 2.36, 0.94, 1.52, 2.12, 3.98, 2.16, 1.35, 1.67, 2.39,
       1.67, 3.74, 2.16, 1.47, 3.7 , 3.03, 1.73, 5.19, 1.33, 1.45, 4.43,
       4.31, 1.19, 1.73, 2.02, 0.98, 1.73, 4.72, 1.6 , 4.36, 1.63, 1.81,
       0.99, 1.75, 2.08, 0.9 , 1.95, 2.59, 1.75, 1.87, 0.94, 1.88, 0.89,
       1.35, 1.73, 5.51, 1.9 , 1.87, 1.71, 1.51, 3.3 , 3.59, 1.48, 3.43,
       2.45, 1.81, 1.97, 3.84, 5.65, 1.21, 1.78, 2.4 , 4.95])

## 2. Look at the distribution of two features from one of the wineries

Our goal is to plot the distribution of two features from a particular winery. We will use several helper functions for this. It is worth understanding each of these.

The first helper function fits a Gaussian to a data set, restricting attention to specified features.
It returns the mean and covariance matrix of the Gaussian.

In [4]:
# Fit a Gaussian to a data set using the selected features
def fit_gaussian(x, features):
    mu = np.mean(x[:,features], axis=0)
    covar = np.cov(x[:,features], rowvar=0, bias=1)
    return mu, covar

For example, let's look at the Gaussian we get for winery 1, using features 0 ('alcohol') and 6 ('flavanoids').

In [5]:
f1 = 0
f2 = 6
label = 1
mu, covar = fit_gaussian(trainx[trainy==label,:], [f1,f2])
print "Mean:\n" + str(mu)
print "Covariance matrix:\n" + str(covar)

Mean:
[13.78534884  2.99627907]
Covariance matrix:
[[0.23325279 0.07526874]
 [0.07526874 0.15240941]]


Next, we will construct a routine for displaying points sampled from a two-dimensional Gaussian, as well as a few contour lines. Part of doing this involves deciding what range to use for each axis. We begin with a little helper function that takes as input an array of numbers (values along a single feature) and returns the range in which these numbers lie.

In [6]:
# Find the range within which an array of numbers lie, with a little buffer
def find_range(x):
    lower = min(x)
    upper = max(x)
    width = upper - lower
    lower = lower - 0.2 * width
    upper = upper + 0.2 * width
    return lower, upper

Next we define a routine that plots a few contour lines of a given two-dimensional Gaussian.
It takes as input:
* `mu`, `cov`: the parameters of the Gaussian
* `x1g`, `x2g`: the grid (along the two axes) at which the density is to be computed
* `col`: the color of the contour lines

In [7]:
def plot_contours(mu, cov, x1g, x2g, col):
    rv = multivariate_normal(mean=mu, cov=cov)
    z = np.zeros((len(x1g),len(x2g)))
    for i in range(0,len(x1g)):
        for j in range(0,len(x2g)):
            z[j,i] = rv.logpdf([x1g[i], x2g[j]]) 
    sign, logdet = np.linalg.slogdet(cov)
    normalizer = -0.5 * (2 * np.log(6.28) + sign * logdet)
    for offset in range(1,4):
        plt.contour(x1g,x2g,z, levels=[normalizer - offset], colors=col, linewidths=2.0, linestyles='solid')

The function **two_features_plot** takes an input two features and a label, and displays the distribution for the specified winery and pair of features.

The first line allows you to specify the parameters interactively using sliders.

In [8]:
@interact_manual( f1=IntSlider(0,0,12,1), f2=IntSlider(6,0,12,1), label=IntSlider(1,1,3,1) )
def two_features_plot(f1,f2,label):
    if f1 == f2: # we need f1 != f2
        print "Please choose different features for f1 and f2."
        return  
    
    # Set up plot
    x1_lower, x1_upper = find_range(trainx[trainy==label,f1])
    x2_lower, x2_upper = find_range(trainx[trainy==label,f2])
    plt.xlim(x1_lower, x1_upper) # limit along x1-axis
    plt.ylim(x2_lower, x2_upper) # limit along x2-axis
    
    # Plot the training points along the two selected features
    plt.plot(trainx[trainy==label, f1], trainx[trainy==label, f2], 'ro')

    # Define a grid along each axis; the density will be computed at each grid point
    res = 200 # resolution
    x1g = np.linspace(x1_lower, x1_upper, res)
    x2g = np.linspace(x2_lower, x2_upper, res)

    # Now plot a few contour lines of the density
    mu, cov = fit_gaussian(trainx[trainy==label,:], [f1,f2])
    plot_contours(mu, cov, x1g, x2g, 'k')
    
    # Finally, display
    plt.xlabel(featurenames[f1], fontsize=14, color='red')
    plt.ylabel(featurenames[f2], fontsize=14, color='red')
    plt.title('Class ' + str(label), fontsize=14, color='blue')
    plt.show()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2YxJywgbWF4PTEyKSwgSW50U2xpZGVyKHZhbHVlPTYsIGRlc2NyaXB0aW9uPXUnZjInLCBtYXjigKY=


## 3. Fit a Gaussian to each class

We now define a function that will fit a Gaussian generative model to the three classes, restricted to a given list of features. The function returns:
* `mu`: the means of the Gaussians, one per row
* `covar`: covariance matrices of each of the Gaussians
* `pi`: list of three class weights summing to 1

In [12]:
# Assumes y takes on values 1,2,3
# mu is a two-dimension vector since we have two input feature. first dimension is label, second dimension is feature
# covar is a three dimensional vector . the second and third dimension is entries in the covar matrix
def fit_generative_model(x, y, features):
    k = 3 # number of classes
    d = len(features) # number of features
    mu = np.zeros((k+1,d)) # list of means
    covar = np.zeros((k+1,d,d)) # list of covariance matrices
    pi = np.zeros(k+1) # list of class weights
    for label in range(1,k+1):
        indices = (y==label)
        mu[label,:], covar[label,:,:] = fit_gaussian(x[indices,:], features)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, covar, pi

In [18]:
mu = np.zeros((3+1,2)) # list of means
covar = np.zeros((3+1,2,2)) # list of covariance matrices
indices = (trainy==3)
mu[3,:], covar[3,:,:] = fit_gaussian(trainx[indices,:], [1,6])
covar

array([[[ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 1.36014105, -0.1508832 ],
        [-0.1508832 ,  0.07375317]]])

Now we will plot the three Gaussians.

In [10]:
@interact_manual( f1=IntSlider(0,0,12,1), f2=IntSlider(6,0,12,1) )
def three_class_plot(f1,f2):
    if f1 == f2: # we need f1 != f2
        print "Please choose different features for f1 and f2."
        return  
    
    # Set up plot
    x1_lower, x1_upper = find_range(trainx[:,f1])
    x2_lower, x2_upper = find_range(trainx[:,f2])
    plt.xlim(x1_lower, x1_upper) # limit along x1-axis
    plt.ylim(x2_lower, x2_upper) # limit along x2-axis
    
    # Plot the training points along the two selected features
    colors = ['r', 'k', 'g']
    for label in range(1,4):
        plt.plot(trainx[trainy==label,f1], trainx[trainy==label,f2], marker='o', ls='None', c=colors[label-1])

    # Define a grid along each axis; the density will be computed at each grid point
    res = 200 # resolution
    x1g = np.linspace(x1_lower, x1_upper, res)
    x2g = np.linspace(x2_lower, x2_upper, res)

    # Show the Gaussian fit to each class, using features f1,f2
    mu, covar, pi = fit_generative_model(trainx, trainy, [f1,f2])
    for label in range(1,4):
        gmean = mu[label,:]
        gcov = covar[label,:,:]
        plot_contours(gmean, gcov, x1g, x2g, colors[label-1])

    # Finally, display
    plt.xlabel(featurenames[f1], fontsize=14, color='red')
    plt.ylabel(featurenames[f2], fontsize=14, color='red')
    plt.title('Wine data', fontsize=14, color='blue')
    plt.show()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2YxJywgbWF4PTEyKSwgSW50U2xpZGVyKHZhbHVlPTYsIGRlc2NyaXB0aW9uPXUnZjInLCBtYXjigKY=


## 4. Predict labels for the test points

How well we can predict the class (1,2,3) based just on these two features?

We start with a testing procedure that is analogous to what we developed in the 1-d case.

In [19]:
# Now test the performance of a predictor based on a subset of features
@interact( f1=IntSlider(0,0,12,1), f2=IntSlider(6,0,12,1) )
def test_model(f1, f2):
    if f1 == f2: # need f1 != f2
        print "Please choose different features for f1 and f2."
        return  
    features= [f1,f2]
    mu, covar, pi = fit_generative_model(trainx, trainy, features)
    
    k = 3 # Labels 1,2,...,k
    nt = len(testy) # Number of test points
    score = np.zeros((nt,k+1))
    for i in range(0,nt):
        for label in range(1,k+1):
            score[i,label] = np.log(pi[label]) + \
            multivariate_normal.logpdf(testx[i,features], mean=mu[label,:], cov=covar[label,:,:])
    predictions = np.argmax(score[:,1:4], axis=1) + 1
    # Finally, tally up score
    errors = np.sum(predictions != testy)
    print "Test error using feature(s): ",
    for f in features:
        print "'" + featurenames[f] + "'" + " ",
    print
    print "Errors: " + str(errors) + "/" + str(nt)# Now test the performance of a predictor based on a subset of features

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2YxJywgbWF4PTEyKSwgSW50U2xpZGVyKHZhbHVlPTYsIGRlc2NyaXB0aW9uPXUnZjInLCBtYXjigKY=


### <font color="magenta">Fast exercise 1</font>

Different pairs of features yield different test errors.
* What is the smallest achievable test error?
* Which pair of features achieves this minimum test error?

*Make a note of your answers to these questions, as you will need to enter them as part of this week's assignment.*

## 5. The decision boundary 

The function **show_decision_boundary** takes as input two features, builds a classifier based only on these two features, and shows a plot that contains both the training data and the decision boundary.

To compute the decision boundary, a dense grid is defined on the two-dimensional input space and the classifier is applied to every grid point. The built-in `pyplot.contour` function can then be invoked to depict the boundary.

In [56]:
def show_decision_boundary(f1,f2):
    # Fit Gaussian to each class
    mu, covar, pi = fit_generative_model(trainx, trainy, [f1,f2])
    
    # Set up dimensions of plot
    x1_lower, x1_upper = find_range(trainx[:,f1])
    x2_lower, x2_upper = find_range(trainx[:,f2])
    #plt.xlim([x1_lower,x1_upper])
    #plt.ylim([x2_lower,x2_upper])

    # Plot points in training set
    colors = ['r', 'k', 'g']
    #for label in range(1,4):
        #plt.plot(trainx[trainy==label,f1], trainx[trainy==label,f2], marker='o', ls='None', c=colors[label-1])

    # Define a dense grid; every point in the grid will be classified according to the generative model
    res = 200
    x1g = np.linspace(x1_lower, x1_upper, res)
    x2g = np.linspace(x2_lower, x2_upper, res)

    # Declare random variables corresponding to each class density
    # 多元正太分布
    random_vars = {}
    for label in range(1,4):
        random_vars[label] = multivariate_normal(mean=mu[label,:],cov=covar[label,:,:])

    # Classify every point in the grid; these are stored in an array Z[]
    Z = np.zeros((len(x1g), len(x2g)))
    for i in range(0,len(x1g)):
        for j in range(0,len(x2g)):
            scores = []
            for label in range(1,4):
                scores.append(np.log(pi[label]) + random_vars[label].logpdf([x1g[i],x2g[j]]))
            Z[i,j] = np.argmax(scores) + 1

    # Plot the contour lines
    #plt.contour(x1g,x2g,Z.T,3,cmap='seismic')
    
    # Finally, show the image
    #plt.xlabel(featurenames[f1], fontsize=14, color='red')
    #plt.ylabel(featurenames[f2], fontsize=14, color='red')
    #plt.show()
    return Z

In [28]:
mu, covar, pi = fit_generative_model(trainx, trainy, [0,6])
random_vars = {}
random_vars[1] = multivariate_normal(mean=mu[1,:],cov=covar[1,:,:])

Let's use the function above to draw the decision boundary using features 0 ('alcohol') and 6 ('flavanoids').

In [58]:
a = show_decision_boundary(0,6)
a.T

array([[2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 3., 3., 3.],
       ...,
       [2., 2., 2., ..., 1., 1., 1.],
       [2., 2., 2., ..., 1., 1., 1.],
       [2., 2., 2., ..., 1., 1., 1.]])

In [68]:
def show_decision_boundary(f1,f2):
    # Fit Gaussian to each class
    mu, covar, pi = fit_generative_model(trainx, trainy, [f1,f2])
    
    # Set up dimensions of plot
    x1_lower, x1_upper = find_range(trainx[:,f1])
    x2_lower, x2_upper = find_range(trainx[:,f2])
    #plt.xlim([x1_lower,x1_upper])
    #plt.ylim([x2_lower,x2_upper])

    # Plot points in training set
    colors = ['r', 'k', 'g']
    for label in range(1,4):
        plt.plot(trainx[trainy==label,f1], trainx[trainy==label,f2], marker='o', ls='None', c=colors[label-1])

    # Define a dense grid; every point in the grid will be classified according to the generative model
    res = 200
    x1g = np.linspace(x1_lower, x1_upper, res)
    x2g = np.linspace(x2_lower, x2_upper, res)

    # Declare random variables corresponding to each class density
    # 多元正太分布
    random_vars = {}
    for label in range(1,4):
        random_vars[label] = multivariate_normal(mean=mu[label,:],cov=covar[label,:,:])

    # Classify every point in the grid; these are stored in an array Z[]
    Z = np.zeros((len(x1g), len(x2g)))
    for i in range(0,len(x1g)):
        for j in range(0,len(x2g)):
            scores = []
            for label in range(1,4):
                scores.append(np.log(pi[label]) + random_vars[label].logpdf([x1g[i],x2g[j]]))
            Z[i,j] = np.argmax(scores) + 1

    # Plot the contour lines
    plt.contour(x1g,x2g,Z.T,3,cmap='seismic')
    
    #Finally, show the image
    plt.xlabel(featurenames[f1], fontsize=14, color='red')
    plt.ylabel(featurenames[f2], fontsize=14, color='red')
    plt.show()


In [ ]:
show_decision_boundary(0,6)

### <font color="magenta">Fast exercise 2</font>

Can you add interactive sliders to function **show_decision_boundary**?

### <font color="magenta">Fast exercise 3</font>

Produce a plot similar to that of **show_decision_boundary**, but in which just the **test** data is shown.
Look back at your answer to *Fast exercise 1*. Is it corroborated by your plot? Are the errors clearly visible?